In [1]:
import pandas as pd

In [48]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [13]:
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/intact/3_A_Lys.xlsx'
df_intact_wild1 = load_data(lys_path)
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/intact/4_A_Lys_A.xlsx'
df_intact_alkb1 = load_data(lys_path)
df_intact_wild1.shape, df_intact_alkb1.shape

((1435, 5), (1404, 5))

In [49]:
_, df_intact_alkb2_only = diff_dfs(df_intact_wild2, df_intact_alkb2)
df_intact_alkb2_only.shape

(418, 5)

In [14]:
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/intact/9_B_Lys.xlsx'
df_intact_wild2 = load_data(lys_path)
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/intact/10_B_Lys_A.xlsx'
df_intact_alkb2 = load_data(lys_path)
df_intact_wild2.shape, df_intact_alkb2.shape

((1391, 5), (1401, 5))

In [78]:
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Lys_FA_15.xlsx'
df_wild1 = load_data(lys_path)
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/16_A_Lys_A.xlsx'
df_alkb1 = load_data(lys_path)
df_wild1.shape, df_alkb1.shape

((6382, 5), (5217, 5))

In [80]:
df_wild1 = df_wild1[df_wild1.Mass>1500]
df_alkb1 = df_alkb1[df_alkb1.Mass>1500]
df_wild1.shape, df_alkb1.shape

((4912, 5), (4079, 5))

In [83]:
_, df_alkb1_only = diff_dfs(df_wild1, df_alkb1)
df_alkb1_only.shape

(2469, 5)

In [7]:
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/21_B_Lys.xlsx'
df_wild2 = load_data(lys_path)
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/22_B_Lys_A.xlsx'
df_alkb2 = load_data(lys_path)
df_wild2.shape, df_alkb2.shape

((4401, 5), (4783, 5))

In [9]:
df_wild2 = df_wild2[df_wild2.Mass>1500]
df_alkb2 = df_alkb2[df_alkb2.Mass>1500]

In [60]:
df_comm_wild2, df_comm_alkb2 = comm_dfs(df_wild2, df_alkb2)
df_comm_wild2.shape, df_comm_alkb2.shape

((1332, 5), (1337, 5))

In [33]:
df_wild2_top = df_wild2.sort_values('Vol', ascending=False).iloc[:1000]
df_alkb2_top = df_alkb2.sort_values('Vol', ascending=False).iloc[:1000]

In [79]:
plotly_zone(df_wild1)
plotly_zone(df_alkb1)
df_wild1.shape, df_alkb1.shape

((6382, 5), (5217, 5))

In [18]:
dfm_comm_alkb2 = match_dfs(df_wild2, df_alkb2)
dfm_comm_wild2 = match_dfs(df_alkb2, df_wild2)
dfm_only_alkb2 = df_alkb2.drop(dfm_comm_alkb2.index)
dfm_only_wild2 = df_wild2.drop(dfm_comm_wild2.index)

In [94]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
# params = [(df_intact_wild2, df_intact_alkb2_only, shift) for shift in np.arange(-64, 64, 1)]
params = [(df_wild1, df_alkb1, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_comm_wild2, df_comm_wild2, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_wild2, df_alkb2, shift) for shift in np.arange(1, 100, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [95]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [89]:
df_shift_hits.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

,shift,hit
0,-14,580
1,1,544
2,-1,521
3,2,494
4,-12,493
5,-2,479
6,19,478
7,-19,477
8,-37,474
9,12,472


In [90]:
df_sh1 = df_shift_hits.copy()
# df_sh2 = df_shift_hits.copy()

In [64]:
df_sample = df_comm_wild2.copy()

In [65]:
shift = Na
df_shift_na_l = match_dfs(df_sample, df_sample, shift=-1*shift)
df_shift_na_r = match_dfs(df_sample, df_sample, shift=shift)
shift = K
df_shift_k_l = match_dfs(df_sample, df_sample, shift=-1*shift)
df_shift_k_r = match_dfs(df_sample, df_sample, shift=shift)
df_shift_na_l.shape, df_shift_na_r.shape, df_shift_k_l.shape, df_shift_k_r.shape

((156, 5), (157, 5), (186, 5), (186, 5))

In [91]:
shift = -1*M
df_shift_l = match_dfs(df_sample, df_sample, shift=-1*shift)
df_shift_r = match_dfs(df_sample, df_sample, shift=shift)

In [56]:
shift = M
dfm_l = match_dfs(df_wild2, dfm_only_alkb2, shift=-1*shift)
dfm_r = match_dfs(dfm_only_alkb2, df_wild2, shift=shift)

In [92]:
plotly_zones(df_shift_l, df_shift_r)

In [93]:
bcr = base_calling_random(df_shift_l, base_only=False)
plotly_basecalling(*bcr)
bcr = base_calling_random(df_shift_r, base_only=False)
plotly_basecalling(*bcr)

mass_x (114,) mass_y (114,)


mass_x (114,) mass_y (114,)
